# Vector Autoregression (VAR)

In [1]:
from IPython.display import display, Markdown
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error

# Charging station Dataset

In [2]:
# Load in the dataset
full_data =  pd.read_csv('C:/Users/Sirine/Desktop/Maseer/201801a.txt',sep='\t', encoding="utf-8" ,names = ["Datetime","Time","ChargePointID","ChargePointType","Status","Latitude_Longitude","Address",    "Latitude","Longitude"])


In [3]:
# Clean empty rows
full_data = full_data.dropna(axis=0)

In [4]:
#convert Datetime to str to combine date (Datetime) and time (Time)
full_data.Datetime = full_data.Datetime.astype(str)

#convert Time to str to combine date (Datetime) and time (Time)
full_data.Time = full_data.Time.astype(str)
#add Time to Datetime column
full_data['Datetime'] = full_data['Datetime'].str.cat(full_data['Time'].values.astype(str), sep='')
#now I don't need the Time column anymone (since it was added to the Datetime column)
 

full_data['Date'] = full_data.Datetime.str.slice(0,8)
full_data['Time'] = full_data.Datetime.str.slice(8)
full_data['Time'] = full_data['Time'].apply(lambda val: '{:0>4}'.format(val))
full_data['Datetime'] = full_data['Date'] + " " + full_data["Time"]

#now let's finally convert Datetime to type datetime in column FormattedDatetime 

full_data['FormattedDatetime'] = pd.to_datetime(full_data.Datetime,format="%Y%m%d %H%M")
del full_data['Datetime']
del full_data['Date']
del full_data['Time']

full_data.head()

,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude,FormattedDatetime
0,CP:C6FD3,StandardType2,OOS,"-6.933468,52.836332","Public Car Park, Kennedy Street, Carlow Town, ...",-6.933468,52.836332,2018-01-01 00:01:00
1,CP:C7WLM,StandardType2,OOC,"-6.972487,53.917571","Main Street, Bailieborough, County Cavan",-6.972487,53.917571,2018-01-01 00:01:00
2,CP:C4FVM,StandardType2,OOS,"-9.346292,52.928911","Public Car Park, N67/Milltown Malbay Road, Lah...",-9.346292,52.928911,2018-01-01 00:01:00
3,CP:C6Q3G,StandardType2,OOC,"-8.889570,51.620831","Deasy's Public Car Park, Park Road (N71), Clon...",-8.889570,51.620831,2018-01-01 00:01:00
4,CP:C6J53,StandardType2,Part,"-8.473326,51.896298","51 South Mall, Cork City, County Cork",-8.473326,51.896298,2018-01-01 00:01:00


## Multivariate Time Series (MTS)

A <b>multivariate time series</b> has more than one time-dependent variable just like our dataset. Each variable depends not only on its past values but also has some dependency on other variables. This dependency is used for forecasting future values.

### to_datetime()

In [5]:

status_dummies = pd.get_dummies(full_data.Status).astype(int)
status_dummies.head()

,OOC,OOS,Occ,Part
0,0,1,0,0
1,1,0,0,0
2,0,1,0,0
3,1,0,0,0
4,0,0,0,1


In [6]:
#let's reappend the status features to the original dataset
full_data = pd.concat(
    [full_data,status_dummies],
    axis=1) #remember that concatenating columns means axis=1!
#full_data.drop('Status',inplace=True,axis=1) --> this would drop the Status column, but we will want that
full_data.head()

,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude,FormattedDatetime,OOC,OOS,Occ,Part
0,CP:C6FD3,StandardType2,OOS,"-6.933468,52.836332","Public Car Park, Kennedy Street, Carlow Town, ...",-6.933468,52.836332,2018-01-01 00:01:00,0,1,0,0
1,CP:C7WLM,StandardType2,OOC,"-6.972487,53.917571","Main Street, Bailieborough, County Cavan",-6.972487,53.917571,2018-01-01 00:01:00,1,0,0,0
2,CP:C4FVM,StandardType2,OOS,"-9.346292,52.928911","Public Car Park, N67/Milltown Malbay Road, Lah...",-9.346292,52.928911,2018-01-01 00:01:00,0,1,0,0
3,CP:C6Q3G,StandardType2,OOC,"-8.889570,51.620831","Deasy's Public Car Park, Park Road (N71), Clon...",-8.889570,51.620831,2018-01-01 00:01:00,1,0,0,0
4,CP:C6J53,StandardType2,Part,"-8.473326,51.896298","51 South Mall, Cork City, County Cork",-8.473326,51.896298,2018-01-01 00:01:00,0,0,0,1


In [7]:

CS1=full_data.loc[full_data['ChargePointID'] == 'CP:C8PJ7'] 




In [8]:

CS1AVAIL = []
current = []
start = '2018-01-01 15:20:00'
time = pd.to_datetime(start)
#count = 0
while time <= pd.to_datetime('2018-1-31 19:52:00'):
    current.append('CP:C8PJ7') #ChargePointID
    current.append('StandardType2') #ChargePointType
    current.append('A') #Status = available
    current.append('Q-Park Multi-Storey Car Park, Sean McDermott Street, Dublin 1, County Dublin') #Address
    current.append(float(-6.258432)) #Latitude
    current.append(float(53.352022)) #citude
    current.append(pd.to_datetime(time)) #Datetime
    current.append(0) #OOC
    current.append(0) #OOS
    current.append(0) #Occ
    current.append(0) #Part
    CS1AVAIL.append(current)
    current = []
    time = pd.to_datetime(time) + pd.Timedelta(minutes=5)
   
 
 
 


In [9]:


#turn list into df
CS1AVAIL = pd.DataFrame(CS1AVAIL, columns=['ChargePointID',
                                                         'ChargePointType',
                                                         'Status',
                                                         'Address',
                                                         'Latitude',
                                                         'Longitude',
                                                         'FormattedDatetime',
                                                         'OOC',
                                                         'OOS',
                                                         'Occ',
                                                         'Part'])

 


 

In [10]:



"""I was getting an error when trying to update cliftonAVAIL with clifton,
so I reset the index of clifton, which fixed the problem"""
CS1 = CS1.reset_index()  
CS1AVAIL.update(CS1) 

In [11]:

CS1AVAIL['Status_Number'] = (CS1AVAIL.Status != 'A').astype(int)
  
CS1AVAIL.head()


,ChargePointID,ChargePointType,Status,Address,Latitude,Longitude,FormattedDatetime,OOC,OOS,Occ,Part,Status_Number
0,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
1,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:25:00,0.0,0.0,0.0,1.0,1
2,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:30:00,0.0,0.0,0.0,1.0,1
3,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:20:00,0.0,0.0,0.0,1.0,1
4,CP:C8PJ7,StandardType2,Part,"Q-Park Multi-Storey Car Park, Sean McDermott S...",-6.258432,53.352022,2018-01-01 15:35:00,0.0,0.0,0.0,1.0,1


In [12]:

CS1AVAIL.drop('ChargePointID',inplace=True,axis=1) 
CS1AVAIL.drop('ChargePointType',inplace=True,axis=1) 
CS1AVAIL.drop('Status',inplace=True,axis=1) 
CS1AVAIL.drop('Latitude',inplace=True,axis=1) 
CS1AVAIL.drop('Longitude',inplace=True,axis=1)  
CS1AVAIL.drop('Address',inplace=True,axis=1) 
CS1AVAIL.drop('Status_Number',inplace=True,axis=1)  

CS1AVAIL.head()

,FormattedDatetime,OOC,OOS,Occ,Part
0,2018-01-01 15:20:00,0.0,0.0,0.0,1.0
1,2018-01-01 15:25:00,0.0,0.0,0.0,1.0
2,2018-01-01 15:30:00,0.0,0.0,0.0,1.0
3,2018-01-01 15:20:00,0.0,0.0,0.0,1.0
4,2018-01-01 15:35:00,0.0,0.0,0.0,1.0


In [13]:

CS1AVAIL = CS1AVAIL.set_index('FormattedDatetime')


## Stationarity

A stationary time series will more often than not give us a better set of predictions. Similar to the <b>Augmented Dickey-Fuller test</b> for univariate series, we have <b>Johansen’s test</b> for checking the stationarity of any multivariate time series data. Since the test works for only 12 variables, I have randomly dropped in the next iteration, I would drop another and check the eigenvalues.

### coint_johansen()

`statsmodels.tsa.vector_ar.vecm.coint_johansen(endog, det_order, k_ar_diff)` - perform the Johansen cointegration test for determining the cointegration rank of a VECM. 
* <b>endog</b> - the data with presample.
* <b>det_order: int</b> -
  * <b>-1</b> - no deterministic terms.
  * <b>0</b> - constant term.
  * <b>1</b> - linear trend.
* <b>k_ar_diff: int, nonnegative</b> - number of lagged differences in the model.

In [14]:
#from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [15]:
#johan_test_temp = data.drop(['CO(GT)'], axis=1)
#coint_johansen(johan_test_temp, -1, 1).eig

## Train-Validation Split

Creating a validation set for time series problems is tricky because we have to take into account the time component. One cannot directly use the `train_test_split` or k-fold validation since this will disrupt the pattern in the series. The validation set should be created considering the date and time values.

In [16]:
train = CS1AVAIL[:int(0.8*(len(CS1AVAIL)))]
valid = CS1AVAIL[int(0.8*(len(CS1AVAIL))):]

## Vector Autoregression (VAR)

In a <b>VAR</b> model, each variable is a linear function of the past values of itself and the past values of all the other variables. Unlike AR, VAR is able to understand and use the relationship between several variables. This is useful for describing the dynamic behavior of the data and also provides better forecasting results. Additionally, implementing VAR is as simple as using any other univariate technique.

In [17]:
from statsmodels.tsa.vector_ar.var_model import VAR

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### VAR()

`statsmodels.tsa.vector_ar.var_model.VAR(endog, exog=None, dates=None, freq=None, missing='none')` - fit VAR(p) process and do lag order selection.
\begin{equation*} 
y_t = A_1y_{t-1} + ... + A_py_{t-p} + u_t
\end{equation*}
* <b>endog</b> - 2-d endogenous response variable. The independent variable.
* <b>exog</b> - 2-d exogenous variable.
* <b>dates</b> - must match number of rows of endog

In [18]:
# Fit the model
model = VAR(endog=train)
model_fit = model.fit()

# Make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [19]:
# Convert array to DataFrame
cols = CS1AVAIL.columns
pred = pd.DataFrame(index=range(0, len(prediction)), columns=[cols])

for j in range(0, len(cols)):
    for i in range(0, len(prediction)):
        pred.iloc[i][j] = prediction[i][j]

In [20]:
 
pred = pred.dropna(axis = 0, how ='any')  


In [21]:
# RMSE
for i in cols:
    print('RMSE value for', i, 'is:', sqrt(mean_squared_error(pred[i], valid[i])))

RMSE value for OOC is: 6.902355878913089e-13
RMSE value for OOS is: 0.0
RMSE value for Occ is: 2.0032042855947532e-13
RMSE value for Part is: 2.7677889792880927e-12


In [22]:
# Make final predictions
model = VAR(endog=CS1AVAIL)
model_fit = model.fit()
 

In [23]:

yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[  1.00292092e-15   0.00000000e+00   5.19189356e-16   1.73625442e-15]]
